In [ ]:
import sys
from math import *

In [ ]:
import logging
logger = logging.getLogger("geant4")

In [ ]:
import Geant4
from Geant4 import cm, mm, MeV, GeV, TeV, G4ThreeVector

In [ ]:
import ROOT

### Access run manager global

In [ ]:
manager = Geant4.gRunManager

### Define geometry and materials initialization callback class

In [ ]:
class DetectorConstructor(Geant4.G4VUserDetectorConstruction):
    def __init__(self):
        super().__init__()
        
        # Create world (box => logical => placement)
        #   solid
        world_box   = Geant4.G4Box("WorldBox", 10*cm, 10*cm, 25*cm)
        #   logical
        vacuum = Geant4.G4NistManager.Instance().FindOrBuildMaterial("G4_Galactic")
        self.world = Geant4.G4LogicalVolume(world_box, vacuum, "World", None, None, None, True)
        #   placement
        r0 = G4ThreeVector(0, 0, 0)
        self.world_placement = Geant4.G4PVPlacement(None, r0, self.world, "World", None, False, 0)
        
        # Create gold cube
        #   solid
        self.calorimeter_box   = Geant4.G4Box("CalorBox", 5*cm, 5*cm, 5*mm)
        #   logical
        gold = Geant4.G4NistManager.Instance().FindOrBuildMaterial("G4_Au")
        assert gold is not None
        self.calorimeter = Geant4.G4LogicalVolume(self.calorimeter_box, gold, "Calorimeter",
                                                          None, None, None, True)
        #   placement
        r1 = G4ThreeVector(0, 0, 10.5*cm)
        self.calorimeter_placement = Geant4.G4PVPlacement(None, r1, self.calorimeter, "Calorimeter",
                                                          self.world, False, 0)

    def Construct(self):
        logger.info("construct detector")
        return self.world_placement

### Define physics list

In [ ]:
PhysicsList = Geant4.FTFP_BERT

### Define particles primary generator

In [ ]:
class PrimaryGenerator(Geant4.G4VUserPrimaryGeneratorAction):

    def __init__(self):
        super().__init__()

        self.energy         = 1.0*GeV
        self.energySpread   = 0.2*GeV

        self.z          =  0.0*mm

        self.gun   = Geant4.G4ParticleGun()

        # set particle type
        self.particle = Geant4.G4ParticleTable.GetParticleTable().FindParticle("gamma")
        assert self.particle is not None
        self.gun.SetParticleDefinition(self.particle)


    def GeneratePrimaries(self, anEvent):

        # set source position
        p = Geant4.G4ThreeVector(0.0, 0.0, self.z)
        self.gun.SetParticlePosition(p)


        # get full energy normal distributed
        energy = ROOT.gRandom.Gaus(self.energy, self.energySpread) 
        if energy < 0.0: energy = 0.0

        # set kinetic (!) energy
        self.gun.SetParticleEnergy(energy - - self.particle.GetPDGMass())

        # shoot momentum direction uniformly by solid angle and set
        phi   = ROOT.gRandom.Uniform(0.0, 2.0*pi)
        theta = acos(ROOT.gRandom.Uniform(cos(0.03), 1.0))
        pdir  = Geant4.G4ThreeVector(sin(theta)*cos(phi), sin(theta)*sin(phi), cos(theta))

        self.gun.SetParticleMomentumDirection(pdir)

        # create particle
        self.gun.GeneratePrimaryVertex(anEvent)

        logger.debug("generate primary event")


### Define event callback (user action) class

In [ ]:
class EventAction(Geant4.G4UserEventAction):
    def __init__(self, sd):
        super().__init__()
        self.sd = sd

    def BeginOfEventAction(self, event):
        # clean up accumulator
        self.sd.energy = 0.0

    def EndOfEventAction(self, event):
        # fill histogram
        self.sd.histogram.Fill(self.sd.energy)

### Define simple sensitive detector callback class (calorimeter)

In [ ]:
class CalorimeterDetector(Geant4.G4VSensitiveDetector):

    def __init__(self):
        super().__init__()
        self.energy = 0.0
        self.histogram = ROOT.TH1F("he", "Energy", 151, -0.05, 1.5+0.05)
        self.histogram.GetXaxis().SetTitle("MeV")
        self.histogram.GetYaxis().SetTitle("events")

    def ProcessHits (self, step, rohistory):
        deposit = step.GetTotalEnergyDeposit()/GeV
        self.energy += deposit
        return True


## Initialization

In [ ]:
# geometry and materials
constructor = DetectorConstructor()
manager.SetUserInitialization(constructor)

# physics
physics = PhysicsList()
manager.SetUserInitialization(physics)
Geant4.gApplyUICommand("/process/had/verbose 0")
Geant4.gApplyUICommand("/process/em/verbose 0")

# primary generator
generator = PrimaryGenerator()
manager.SetUserAction(generator)

# sensitive detectors
sd = CalorimeterDetector()
constructor.calorimeter.SetSensitiveDetector(sd)

# event initialization callback
eventcallback = EventAction(sd)
manager.SetUserAction(eventcallback)

manager.Initialize()

## Set up visualization parameters and scene contents

In [ ]:
Geant4.gApplyUICommand("/vis/viewer/flush")

#Geant4.gApplyUICommand("/vis/x3dfile/viewHalfAngle 20")
Geant4.gApplyUICommand("/vis/open X3DFILE")

Geant4.gApplyUICommand("/vis/viewer/set/autoRefresh false")
Geant4.gApplyUICommand("/vis/viewer/refresh")

Geant4.gApplyUICommand("/vis/scene/create")
Geant4.gApplyUICommand("/vis/scene/add/volume")


Geant4.gApplyUICommand("/vis/drawVolume")

Geant4.gApplyUICommand("/vis/modeling/trajectories/create/drawByParticleID")
Geant4.gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set gamma white")
Geant4.gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set proton green")
Geant4.gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set e- yellow")
Geant4.gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set e+ blue")

Geant4.gApplyUICommand("/vis/sceneHandler/attach")

Geant4.gApplyUICommand("/tracking/storeTrajectory 1")
Geant4.gApplyUICommand("/vis/scene/add/trajectories")
Geant4.gApplyUICommand("/vis/scene/add/hits")
Geant4.gApplyUICommand("/vis/scene/add/trajectories smooth")

#Geant4.gApplyUICommand("/vis/scene/add/eventID 1")
#Geant4.gApplyUICommand("/vis/scene/add/date")
Geant4.gApplyUICommand("/vis/viewer/set/autoRefresh true")

Geant4.gApplyUICommand("/vis/viewer/set/style surface")
Geant4.gApplyUICommand("/vis/viewer/set/viewpointThetaPhi 60 10")

Geant4.gApplyUICommand("/vis/geometry/set/colour Calorimeter 0 1 0 0 0")
Geant4.gApplyUICommand("/vis/geometry/set/colour World 0 0 0 1 0.6")
#Geant4.gApplyUICommand("/vis/geometry/set/forceWireframe World 0 1")

Geant4.gApplyUICommand("/vis/enable false")

## Cleanup x3d files from old runs

In [ ]:
from os import unlink
from glob import glob
for fdel in glob("g4*.html"):
    unlink(fdel)

## Generate three events with visualization enabled

In [ ]:
Geant4.gApplyUICommand("/vis/enable true")
manager.BeamOn(3)
Geant4.gApplyUICommand("/vis/enable false")

print(glob("g4_*.html"))

## Show generated events

In [ ]:
import IPython.display

In [ ]:
with open("g4_00.html", "r") as x3dfile:
        data = x3dfile.read()
IPython.display.HTML(data)

In [ ]:
with open("g4_01.html", "r") as x3dfile:
        data = x3dfile.read()
IPython.display.HTML(data)

In [ ]:
with open("g4_02.html", "r") as x3dfile:
        data = x3dfile.read()
IPython.display.HTML(data)